In [398]:
import pandas as pd
import numpy as np
import datetime

In [617]:
user = pd.read_csv('user.csv',encoding='utf-8')
user_log = pd.read_csv('user_log.csv',encoding='utf-8')
user_song = pd.read_csv('user_song.csv',encoding='utf-8')
song_info = pd.read_csv('song_info.csv',encoding='utf-8')

In [618]:
user_log.head()

,user_id,log_in_timestamp,log_out_timestamp
0,bf98edd3-fe2b-409a-abd8-311a2109261b,2017/6/5 23:26,2017/6/6 0:21
1,bf98edd3-fe2b-409a-abd8-311a2109261b,2017/6/6 22:26,2017/6/7 0:21
2,bf98edd3-fe2b-409a-abd8-311a2109261b,2017/6/7 9:26,2017/6/7 10:21
3,112b003d-244f-4657-8d19-315ad3ec23f5,2017/6/5 3:11,2017/6/5 17:41
4,112b003d-244f-4657-8d19-315ad3ec23f5,2017/6/6 3:11,2017/6/6 17:41


In [619]:
song_info.head()

,song_id,song_name,song_style,song_lyrics,song_singer,song_hotness,song_year,song_month
0,56138ca287e9434581266693dd581692,演员,流行,简单点 说话的方式简单点\n递进的情绪请省略\n你又不是个演员\n别设计那些情节\n没意见 ...,薛之谦,9,2015,5
1,58dd5f51326745829b2654a9a3dc2e8a,简单爱,流行,说不上为什么 我变得很主动\n若爱上一个人 什么都会值得去做\n我想大声宣布 对你依依不舍\...,周杰伦,7,2001,9
2,ce796432b64e4e8692346bb202a830cb,寻人启事,流行,让我看看 你的照片\n 究竟为什么 你消失不见\n 多数时间 你在哪边\n 会不会疲倦...,徐佳莹,8,2014,6
3,36254f9e158f40a4a2376299a97707f6,身骑白马,流行,我爱谁 跨不过 从来也不觉得错\n自以为 抓着痛 就能往回忆里躲\n偏执相信着 受诅咒的水晶...,徐佳莹,8,2009,5
4,a3590a5a7e9b4725a3cc947e240a1a81,飘向北方,流行,飘向北方 别问我家乡\n高耸古老的城墙 挡不住忧伤\n我飘向北方 家人是否无恙\n肩上沉重的...,王力宏,8,2017,3


In [620]:
user_song.head()

,user_id,song_id,song_in_timestamp,finished,song_source,favorite
0,bf98edd3-fe2b-409a-abd8-311a2109261b,56138ca287e9434581266693dd581692,1496644018,1,1,1.0
1,112b003d-244f-4657-8d19-315ad3ec23f5,58dd5f51326745829b2654a9a3dc2e8a,1496684411,0,2,1.0
2,112b003d-244f-4657-8d19-315ad3ec23f5,ce796432b64e4e8692346bb202a830cb,1496632306,1,3,1.0
3,68a0a10a-8c3d-4b96-92ba-4c74e69198aa,36254f9e158f40a4a2376299a97707f6,1496660910,0,1,1.0
4,496291a0-008f-416a-a095-af6730836355,a3590a5a7e9b4725a3cc947e240a1a81,1496622320,1,2,1.0


In [404]:
def total_num_songs(user_song, user_id):
    # 返回： 指定user_id听的歌曲中去掉重复歌曲的个数
    return len(user_song[user_song['user_id'] == user_id]['song_id'].unique())

In [479]:
def favo_style(user_song, user_id, song_info):
    # 将对应user_id听的歌的id存在list里
    song_list = list(user_song[user_song['user_id'] == user_id]['song_id'])
    # 找出将上述list中出现次数最高的歌的id
    fav_song_id = max(set(song_list), key = song_list.count)
    # 返回最高次数歌的风格
    return song_info[song_info['song_id'] == fav_song_id]['song_style'][0]

In [480]:
favo_style(user_song, 'bf98edd3-fe2b-409a-abd8-311a2109261b', song_info)

'流行'

In [481]:
def build_time_period_feature():# 该函数是一个助手函数，添加时间段特征
    #将开始听歌的时间戳转换为小时
    user_song['song_in_hour'] = pd.to_datetime(user_song['song_in_timestamp'] , unit='s').dt.hour
    #将小时转换为对应的时间段
    shenye = (user_song['song_in_hour'] >= 0) & (user_song['song_in_hour']<= 4)
    qingchen = (user_song['song_in_hour'] > 4) & (user_song['song_in_hour'] <=8)
    shangwu = (user_song['song_in_hour'] > 8) & (user_song['song_in_hour'] <=12)
    xiawu = (user_song['song_in_hour'] > 12) & (user_song['song_in_hour'] <=16)
    huanghun = (user_song['song_in_hour'] > 16) & (user_song['song_in_hour'] <=20)
    wanshang = (user_song['song_in_hour'] > 20) & (user_song['song_in_hour'] <=23)
    
    user_song['song_in_time_period'] = '晚上'
    user_song['song_in_time_period'][shenye] = '深夜'
    user_song['song_in_time_period'][qingchen] = '清晨'
    user_song['song_in_time_period'][shangwu] = '上午'
    user_song['song_in_time_period'][xiawu] = '下午'
    user_song['song_in_time_period'][huanghun] = '黄昏'
    user_song['song_in_time_period'][wanshang] = '晚上'
    
def favo_time_period(user_song, user_id):
    #如果还没添加时间段这个特征，先添加
    if 'song_in_time_period' not in user_song.columns:
        build_time_period_feature()
    # 将对应user_id听的歌的时间段存在list里
    time_list = list(user_song[user_song['user_id'] == user_id]['song_in_time_period'])
    # 返回上述list里出现最多的时间段名
    fav_time = max(set(time_list), key = time_list.count)
    return fav_time

In [482]:
favo_time_period(user_song, 'bf98edd3-fe2b-409a-abd8-311a2109261b')

'下午'

In [621]:
def comment(user, user_id):
    if user[user['user_id'] == user_id]['comment_time'][0] <= 1:
        return ['喜欢默默地听', '评论区的狂欢你很少参与.']
    elif (user[user['user_id'] == user_id]['comment_time'][0] > 1 
          and user[user['user_id'] == user_id]['comment_time'][0] < 3):
        return ['喜欢分享自己的感受', '你经常在评论区分享听歌感受.']
    else:
        return ['热爱分享', '评论区里藏着你的许多回忆.']

In [622]:
 comment(user, 'bf98edd3-fe2b-409a-abd8-311a2109261b')

['喜欢默默地听', '评论区的狂欢你很少参与.']

In [413]:
def most_freq_words(user_log, user_id, song_info):
    import jieba
    import re
    from collections import Counter
    # 将user_id听过的歌去重以后的歌词全部连起来
    lyrics_list = list(pd.merge(song_info, user_song[user_song['user_id'] == user_id], how = 'right')
                       .drop_duplicates('song_id')['song_lyrics'])
    lyrics = ''.join(lyrics_list)
    # 去除非汉字的数字、英文及符号等
    filtrate = re.compile(u'[^\u4E00-\u9FA5]')
    filtered_lyrics = filtrate.sub(r'', lyrics)
    # 使用Jieba中文分词并转化为list
    seg_list = list(jieba.cut(filtered_lyrics))
    # 返回上述list中出现频率最高的2字词（除去一些停止词）
    count = Counter(seg_list)
    freq_list = count.most_common()
    stopwords = ['的','你','我','人','在','让','了','和','是','就','都','而','及','與','著','或',
                 '一個','一个','沒有','没有','我們','我们','你們','你们','妳們','他們','他们',
                 '她們','是否']
    return [w[0] for w in freq_list if w[0] not in stopwords and len(w[0]) >1][0]

In [414]:
most_freq_words(user_log, 'bf98edd3-fe2b-409a-abd8-311a2109261b', song_info)

'永远'

In [623]:
def build_date_feature():
    # 这是一个助手函数，将时间戳转变为‘2018-01-02’这样的日期形式
    user_song['song_in_date'] = pd.to_datetime(user_song['song_in_timestamp'] , unit='s').dt.date
    
def most_freq_in_one_day(user_song, user_id, song_info):
    # 检查是否已经转变了时间戳，如果没有，就转变
    if 'song_in_date' not in user_song.columns:
        build_date_feature()
    # 找出在一天内user_id 听过次数最多的歌，返回这一天，歌名，演唱者，以及听了多少次
    f_df = (pd.DataFrame(user_song[user_song['user_id'] == user_id]
                         .groupby(['song_in_date','song_id'])
                         .user_id.count())
                        .reset_index()
                        .sort_values('user_id', ascending = False))
    month = f_df.iloc[0]['song_in_date'].month
    day = f_df.iloc[0]['song_in_date'].day
    time = f_df.iloc[0]['user_id']
    song_id = f_df.iloc[0]['song_id']
    song_name = song_info[song_info['song_id'] == song_id].iloc[0]['song_name']
    singer = song_info[song_info['song_id'] == song_id].iloc[0]['song_singer']

    return month, day, song_name, singer, time

In [624]:
most_freq_in_one_day(user_song, 'bf98edd3-fe2b-409a-abd8-311a2109261b', song_info)

(6, 7, '小酒窝', '林俊杰', 3)

In [418]:
def to_datetime():
    user_log['log_out_timestamp'] = pd.to_datetime(user_log['log_out_timestamp'])
    user_log['log_in_timestamp'] = pd.to_datetime(user_log['log_in_timestamp'])
def late_night(user_log, user_id):
    if type(user_log['log_out_timestamp'][0]) is str:
        to_datetime()
    user_log['log_out_date'] = user_log['log_out_timestamp'].dt.date
    condition = (user_log['user_id'] == user_id) & (user_log['log_out_timestamp'].dt.hour >= 0) &(user_log['log_out_timestamp'].dt.hour <= 5)
    return user_log[condition].groupby('log_out_date').user_id.count().sum()

In [419]:
late_night(user_log, 'bf98edd3-fe2b-409a-abd8-311a2109261b')

2

In [547]:
def less_popluar(song_info, user_song, user_id):
    less_pop = song_info[song_info['song_hotness'] <= 5]
    time = pd.merge(less_pop,user_song[user_song['user_id'] == user_id], how = 'left').song_in_timestamp.count()
    if time == 0:
        return '你听过的歌曲都是大热门'
    else:
        f_df = pd.DataFrame(pd.merge(less_pop,user_song[user_song['user_id'] == user_id], how = 'left').groupby('song_id').user_id.count()).reset_index().sort_values('user_id', ascending = False)
        song_id = f_df.iloc[0]['song_id']
        song_name = less_pop[less_pop['song_id'] == song_id].iloc[0]['song_name']
        return '在你的音乐品味中\n也藏着高冷的一面，这首有些小众的'+ '《'+song_name + '》\n'+'在今年你一共听了'+ str(time)+'次'

In [548]:
less_popluar(song_info, user_song, '1dd0d404-4813-45e5-bffc-6324f31daa8b')

'在你的音乐品味中\n也藏着高冷的一面，这首有些小众的《月亮惹的祸》\n在今年你一共听了1次'

In [422]:
def most_freq_song_by_day(song_info, user_song, user_id):
    from collections import Counter
    song_id, time = Counter(user_song[user_song['user_id'] == user_id].groupby(['song_in_date','song_id']).count().reset_index()['song_id']).most_common()[0]
    return song_info[song_info['song_id'] == song_id].iloc[0]['song_name'], time

In [423]:
most_freq_song_by_day(song_info, user_song, '1dd0d404-4813-45e5-bffc-6324f31daa8b')

('月亮惹的祸', 1)

In [426]:
def oldest_song(song_info, user_song, user_id):
    song_name, song_year, song_month = pd.merge(user_song[user_song['user_id'] == user_id].drop_duplicates('song_id'),song_info, how = 'left').sort_values(['song_year','song_month']).iloc[0][['song_name','song_year','song_month']]
    return song_name, song_year, song_month

In [427]:
oldest_song(song_info, user_song, '5c9508b7-b0cd-45d4-bf28-dcefe49ab43a')

('每次都想呼喊你的名字', 1986, 5)

In [428]:
def num_of_recommanded(user_song, user_id):
    mask = (user_song['user_id'] == user_id) & (user_song['song_source'] == 1) & (user_song['favorite'] == 1)
    recom_num = user_song[mask]['song_id'].nunique()
    return recom_num

In [429]:
num_of_recommanded(user_song, '68a0a10a-8c3d-4b96-92ba-4c74e69198aa')

1

In [592]:
def favo_singer(song_info, user_song, user_id):
    user_s_info = pd.merge(user_song[user_song['user_id'] ==  user_id], song_info)
    favo_singer = user_s_info.groupby('song_singer').count().sort_values('user_id', ascending = False).reset_index().iloc[0]['song_singer']
    singer_song_list = user_s_info[user_s_info['song_singer'] == favo_singer].groupby('song_name').count().sort_values('user_id', ascending = False).reset_index()
    f_dict = {'《'+k+'》':v for k, v in zip(singer_song_list['song_name'],singer_song_list['user_id'])}
    time  = sum(f_dict.values())
    f = [k+str(v)+'次\n' for k, v in f_dict.items()]
    return favo_singer, time, f

In [594]:
favo_singer(song_info, user_song, 'bf98edd3-fe2b-409a-abd8-311a2109261b')[2]

['《寻人启事》3次\n', '《言不由衷》2次\n']

In [432]:
user_log.head()

,user_id,log_in_timestamp,log_out_timestamp,log_out_date
0,bf98edd3-fe2b-409a-abd8-311a2109261b,2017-06-05 23:26:00,2017-06-06 00:21:00,2017-06-06
1,bf98edd3-fe2b-409a-abd8-311a2109261b,2017-06-06 22:26:00,2017-06-07 00:21:00,2017-06-07
2,bf98edd3-fe2b-409a-abd8-311a2109261b,2017-06-07 09:26:00,2017-06-07 10:21:00,2017-06-07
3,112b003d-244f-4657-8d19-315ad3ec23f5,2017-06-05 03:11:00,2017-06-05 17:41:00,2017-06-05
4,112b003d-244f-4657-8d19-315ad3ec23f5,2017-06-06 03:11:00,2017-06-06 17:41:00,2017-06-06


In [453]:
def total_hours(user_log, user_id):
    user_log_id = user_log[user_log['user_id'] == user_id]
    total_timedelta = (user_log_id['log_out_timestamp'] - user_log_id['log_in_timestamp']).sum()
    days, seconds = total_timedelta.days, total_timedelta.seconds
    hour = days * 24 + seconds / 3600
    return round(hour)

In [454]:
total_hours(user_log, '112b003d-244f-4657-8d19-315ad3ec23f5')

29

In [612]:
def favorite_songs(song_info, user_song, user_id):
    user_s_info = pd.merge(user_song[user_song['user_id'] ==  user_id], song_info)
    sort_list = user_s_info.groupby(['song_id','song_name','song_singer']).count().sort_values('user_id', ascending = False).reset_index()
    song_name_list = list(sort_list['song_name'])
    singer_list = list(sort_list['song_singer'])
    if len(song_name_list) > 10:
        return ['《'+k+'》 '+v+'\n' for k, v in zip(song_name_list[:10],singer_list[:10])]
    else:
        return ['《'+k+'》 '+v+'\n' for k, v in zip(song_name_list,singer_list)]

In [613]:
favorite_songs(song_info, user_song, '112b003d-244f-4657-8d19-315ad3ec23f5')

['《简单爱》 周杰伦\n', '《寻人启事》 徐佳莹\n']

In [614]:
def net_ease_annual_summary(user, user_log, user_song, song_info, user_id):
    total_num_song = total_num_songs(user_song, user_id)
    fav_style = favo_style(user_song, user_id, song_info)
    fav_time_period = favo_time_period(user_song, user_id)
    comments = comment(user, user_id)
    most_freq_word = most_freq_words(user_log, user_id, song_info)
    most_freqs_in_one_day = most_freq_in_one_day(user_song, user_id, song_info)
    late_nights = late_night(user_log, user_id)
    less_popluars = less_popluar(song_info, user_song, user_id)
    most_freqs_song_by_day = most_freq_song_by_day(song_info, user_song, user_id)
    oldest_songs = oldest_song(song_info, user_song, user_id)
    num_of_recommand = num_of_recommanded(user_song, user_id)
    favo_singers = favo_singer(song_info, user_song, user_id)
    total_hour = total_hours(user_log, user_id)
    favorite_song = favorite_songs(song_info, user_song, user_id)
    
    res = ('这一年里\n你一共在云村听了'+str(total_num_song)+'首歌\n \n你热衷的风格是' 
           +fav_style+'音乐\n喜欢在'+fav_time_period+'听歌\n你'+comments[0]+'\n'+comments[1]
           +'\n\n'+'2017,你听到的最多的歌词是'+'"'+most_freq_word+'"'+'\n\n'+str(most_freqs_in_one_day[0])+'月'
           + str(most_freqs_in_one_day[1])+'日大概是特别的一天\n这一天里\n你把'+ most_freqs_in_one_day[3]+'的'+'《'
           + most_freqs_in_one_day[2]+ '》\n反复听了'+str(most_freqs_in_one_day[4])+'次'+'\n\n这一年,你有'+str(late_nights)+'天\n'
           +'深夜12点后\n仍沉浸在音乐世界\n\n'+ '睡不着的夜晚\n还有网易云音乐的陪伴\n\n'+ less_popluars
           +'\n\n这一年\n在'+ str(most_freqs_song_by_day[1])+'天你都听了'+'《'+ most_freqs_song_by_day[0]
           + '》\n在所有熟悉的旋律中\n你对这首歌最专一\n\n这首'+'《'+oldest_songs[0]+'》'+'是你今年听过年代最久远的歌\n发行于'
           +str(oldest_songs[1])+'年'+str(oldest_songs[2])+'月\n\n这一年，在你收藏的歌\n有'+str(num_of_recommand)
           +'首是网易云音乐精心为你推荐的\n2017, 还是网易云音乐最懂你\n\n'+favo_singers[0]+'\n是你的2017年度歌手\n共听了'
           +str(favo_singers[1]) +'次\n'+ ''.join(favo_singers[2])+'\n这一年\n 你有'+str(total_hour)
           +'小时\n沉浸在音乐的世界\n时光中的音乐\n是不散的宴席\n\n2017\n这些是你最爱的歌\n'+''.join(favorite_song))
    return res

In [615]:
print(net_ease_annual_summary(user, user_log, user_song, song_info,  'bf98edd3-fe2b-409a-abd8-311a2109261b'))

这一年里
你一共在云村听了4首歌
 
你热衷的风格是流行音乐
喜欢在下午听歌
你喜欢默默地听
评论区的狂欢你很少参与.

2017,你听到的最多的歌词是"永远"

6月7日大概是特别的一天
这一天里
你把林俊杰的《小酒窝》
反复听了3次

这一年,你有2天
深夜12点后
仍沉浸在音乐世界

睡不着的夜晚
还有网易云音乐的陪伴

你听过的歌曲都是大热门

这一年
在2天你都听了《演员》
在所有熟悉的旋律中
你对这首歌最专一

这首《小酒窝》是你今年听过年代最久远的歌
发行于2008年10月

这一年，在你收藏的歌
有1首是网易云音乐精心为你推荐的
2017, 还是网易云音乐最懂你

徐佳莹
是你的2017年度歌手
共听了5次
《寻人启事》3次
《言不由衷》2次

这一年
 你有4小时
沉浸在音乐的世界
时光中的音乐
是不散的宴席

2017
这些是你最爱的歌
《小酒窝》 林俊杰
《演员》 薛之谦
《寻人启事》 徐佳莹
《言不由衷》 徐佳莹



In [ ]:
(6, 7, '小酒窝', '林俊杰', 3)